In [ ]:
%run my_import

uri = socket.gethostname()+".local:5555"
#uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"

#uri = "localhost:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))

In [ ]:
import asyncio

import zmq
import zmq.asyncio
import pprint
import json
from protobuf_to_dict import protobuf_to_dict

import sys
sys.path.append('/home/amargan/work/code/ecat_dev/ecat_master_advr/build/protobuf')
import ecat_pdo_pb2
#from ecat_repl import ecat_pdo_pb2

# Define a global variable to store the last received data
last_recv_data = None,None

def process_data(msg_id, data):
    rx_pdo = ecat_pdo_pb2.Ec_slave_pdo()
    rx_pdo.ParseFromString(data)
    pb_dict = protobuf_to_dict(rx_pdo)
    return msg_id,pb_dict

#from utils import get_address
def get_address():
    protocol = "tcp"
    address = "127.0.0.1"
    port = 9601
    return f"{protocol}://{address}:{port}"

async def zsub():
    global last_recv_data
    subscriber = zmq.asyncio.Context().socket(zmq.SUB)
    subscriber.connect(get_address())
    subscriber.subscribe(b"")

    while True:
        msg_id, data = await subscriber.recv_multipart()
        # store the last received data
        last_recv_data = process_data(msg_id, data)
        #pprint.pprint((msg_id, pb_dict))



In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Create a background task for the zsub coroutine
zsub_task = asyncio.create_task(zsub())

In [ ]:
last_recv_data

In [ ]:
pprint.pprint(last_recv_data[1]['header'])

In [ ]:
zsub_task.cancel()